# Compiling all data sources

In [1]:
import numpy as np
import pandas as pd
import requests
import json
import os
opensecretsapi = os.getenv('opensecretsapi')
congressapi = os.getenv('congressapi')
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']

## OpenSecrets: Members

In [37]:
states = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [51]:
opensecrets_members = pd.DataFrame()
for s in states:
    #print(s)
    root = 'https://www.opensecrets.org/api/'
    headers = {'User-Agent': useragent,
              'From': 'jkropko@virginia.edu'}
    params = {'method': 'getLegislators',
              'id': s,
              'output': 'json',
              'apikey':opensecretsapi}
    r = requests.get(root, params=params, headers=headers)
    myjson = json.loads(r.text)
    leg = pd.json_normalize(myjson, record_path=['response', 'legislator'])
    opensecrets_members = pd.concat([opensecrets_members, leg])

In [53]:
opensecrets_members.columns = opensecrets_members.columns.str.replace('@attributes.', '')
opensecrets_members.to_csv('Data/opensecrets_members.csv', index=False)

## https://theunitedstates.io/: Members

In [74]:
url = 'https://theunitedstates.io/congress-legislators/legislators-current.json'
r = requests.get(url, headers=headers)
myjson = json.loads(r.text)
usio_members = pd.json_normalize(myjson)
usio_members.columns = usio_members.columns.str.replace('id.', '')
usio_members.to_csv('Data/usio_members.csv', index=False)

## Congress API: members

In [77]:
root = 'https://api.congress.gov'
params = {'api_key': congressapi}

bio_ids = usio_members['bioguide']
congress_members = pd.DataFrame()
i = 0
for b in bio_ids:
    if (i % 10)==0:
        print(f'Now working on member {i} of {len(bio_ids)}')
    if b != '':
        endpoint = f'/v3/member/{b}'
        r = requests.get(root + endpoint, params=params, headers=headers)
        myjson = json.loads(r.text)
        mem = pd.json_normalize(myjson['member'])
        congress_members = pd.concat([congress_members, mem])
    i += 1

Now working on member 0 of 539


KeyError: 'member'

In [76]:
r.url

'https://api.congress.gov/v3/member/E000246?api_key=6PQeGY3fU5pP9fbxRdG7tKwhgLgOoaG1SCdwqHTg'

In [27]:
congress_members

,bioguideId,birthYear,currentMember,directOrderName,district,firstName,honorificName,invertedOrderName,lastName,officialWebsiteUrl,...,addressInformation.zipCode,cosponsoredLegislation.count,cosponsoredLegislation.url,depiction.attribution,depiction.imageUrl,sponsoredLegislation.count,sponsoredLegislation.url,deathYear,middleName,nickName
0,K000394,1982,True,Andy Kim,3.0,Andy,Mr.,"Kim, Andy",Kim,https://kim.house.gov/,...,20515.0,1199,https://api.congress.gov/v3/member/K000394/cos...,Image courtesy of the Member,https://www.congress.gov/img/member/k000394_20...,86.0,https://api.congress.gov/v3/member/K000394/spo...,NaN,NaN,NaN
0,B001320,1979,True,Laphonza R. Butler,NaN,Laphonza,Ms.,"Butler, Laphonza R.",Butler,https://www.butler.senate.gov/,...,20510.0,2,https://api.congress.gov/v3/member/B001320/cos...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,P000145,1973,True,Alex Padilla,NaN,Alex,Mr.,"Padilla, Alex",Padilla,https://www.padilla.senate.gov,...,20510.0,1094,https://api.congress.gov/v3/member/P000145/cos...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/p000145_20...,190.0,https://api.congress.gov/v3/member/P000145/spo...,NaN,NaN,NaN
0,F000062,1933,True,Dianne Feinstein,NaN,Dianne,Mrs.,"Feinstein, Dianne",Feinstein,NaN,...,20510.0,6727,https://api.congress.gov/v3/member/F000062/cos...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/https://co...,2211.0,https://api.congress.gov/v3/member/F000062/spo...,2023,NaN,NaN
0,W000235,1931,NaN,Ed Weber,9.0,ED,NaN,"Weber, Ed",WEBER,NaN,...,NaN,218,https://api.congress.gov/v3/member/W000235/cos...,NaN,https://www.congress.gov/img/member//assets/no...,5.0,https://api.congress.gov/v3/member/W000235/spo...,2023,NaN,NaN
0,W000038,1936,NaN,Alton R. Waldon Jr.,6.0,ALTON,NaN,"Waldon, Alton R., Jr.",WALDON,NaN,...,NaN,60,https://api.congress.gov/v3/member/W000038/cos...,Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/https://co...,3.0,https://api.congress.gov/v3/member/W000038/spo...,2023,R JR,NaN
0,H001038,1959,True,Brian Higgins,26.0,Brian,Mr.,"Higgins, Brian",Higgins,https://higgins.house.gov,...,20515.0,3541,https://api.congress.gov/v3/member/H001038/cos...,"Image, Congressional Pictorial Directory, 109th.",https://www.congress.gov/img/member/https://co...,132.0,https://api.congress.gov/v3/member/H001038/spo...,NaN,NaN,NaN
0,E000198,1931,NaN,Arlen Erdahl,1.0,ARLEN,NaN,"Erdahl, Arlen",ERDAHL,NaN,...,NaN,562,https://api.congress.gov/v3/member/E000198/cos...,Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/https://co...,28.0,https://api.congress.gov/v3/member/E000198/spo...,2023,INGOLF,NaN
0,C000448,1928,NaN,Dick Clark,NaN,RICHARD,NaN,"Clark, Dick",CLARK,NaN,...,NaN,524,https://api.congress.gov/v3/member/C000448/cos...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/https://co...,69.0,https://api.congress.gov/v3/member/C000448/spo...,2023,CLARENCE,DICK
0,S001192,1960,True,Chris Stewart,2.0,Chris,Mr.,"Stewart, Chris",Stewart,NaN,...,20515.0,1158,https://api.congress.gov/v3/member/S001192/cos...,Image courtesy of the Member,https://www.congress.gov/img/member/https://co...,95.0,https://api.congress.gov/v3/member/S001192/spo...,NaN,NaN,NaN


## Congress API: sponsored legislation

## Congress API: bills

## Congress API: committees/subcommittees

## https://theunitedstates.io/: committee assignments

In [10]:
congressapi

'6PQeGY3fU5pP9fbxRdG7tKwhgLgOoaG1SCdwqHTg'

## OpenSecrets API: members

## OpenSecrets API: contributions

## Voteview: voting similarity matrix

## Voteview: ideology

## MIT Elections: House and Senate